# 1. Objetivo deste Notebook

Este notebook tem como objetivo consolidar os principais resultados obtidos ao longo do projeto e traduzi-los em **conclusões operacionais e estratégicas**, avaliando:

* A viabilidade de prever a qualidade do concentrado de minério a partir de variáveis de processo;
* O desempenho relativo de diferentes abordagens de modelagem;
* As hipóteses formuladas no início do estudo à luz das evidências empíricas;
* As implicações práticas para uso do modelo em um contexto industrial real.

Este notebook **não introduz novos modelos**, mas integra e interpreta os resultados obtidos anteriormente.

# 2. Objetivos do Estudo

Este estudo avaliou a viabilidade do uso de dados operacionais de uma planta de flotação para prever o teor de sílica no concentrado de minério de ferro,\
considerando as restrições reais do processo produtivo, da instrumentação e da forma de coleta do dado de qualidade.\
A seguir, cada objetivo inicial é analisado à luz das evidências empíricas obtidas ao longo do projeto.

## Objetivo 1

*“É possível prever o % de Sílica no Concentrado a cada minuto?”*

**Não alcançado e não factível no cenário atual**\
\
A análise dos dados revelou que o teor de sílica no concentrado:

* possui frequência **horária**;
* é replicado na base de dados para todas as amostras intrahorárias.

Isso implica que **não existe informação nova de target em nível de minuto**, apesar de os sensores fornecerem medições a cada **vinte segundos**, tornando\
inviável qualquer tentativa de previsão real nesse horizonte de tempo. Modelos treinados em granularidade menor do que a do processo de medição apenas\
reproduzem o mesmo valor, sem ganho de informação.

**Conclusão:** a previsão por minuto não é compatível com a natureza dos dados disponíveis e não representa um problema bem definido.

## Objetivo 2

*“Com quantas etapas (horas) de antecedência conseguimos prever o % de Sílica no Concentrado?”*\
\
**Parcialmente alcançado e conceitualmente viável**\
\
A análise temporal mostrou que:

* o target apresenta **alta autocorrelação horária**, com decaimento lento;
* variáveis agregadas em janelas horárias (média e variabilidade) carregam sinal relevante;
* modelos simples conseguem superar o baseline de persistência com **1 hora de antecedência**.

No entanto:

* o ganho sobre o baseline diminui rapidamente com o aumento do horizonte;
* há forte dependência temporal, indicando que grande parte da previsibilidade vem da própria inércia do processo.

**Conclusão:** previsões com **1 hora de antecedência são viáveis**; horizontes maiores exigiriam enriquecimento de dados ou modelagem mais sofisticada.

## Objetivo 3

*“É possível prever o % de Sílica no Concentrado sem utilizar o % de Ferro no Concentrado?”*\
\
**Alcançado**\
\
A exclusão explícita do % de Ferro no Concentrado não inviabilizou a modelagem, mais do que isso: preservou capacidade preditiva superior\
ao baseline e evitou vazamento de informação e redundância conceitual.\
A decisão mostrou-se coerente com a alta correlação estrutural entre ferro e sílica no concentrado e com um cenário operacional realista,\
no qual apenas variáveis de processo estão disponíveis antes da análise laboratorial.

**Conclusão:** o % de Sílica pode ser previsto de forma independente, usando apenas variáveis operacionais e temporais.

### Objetivo Geral

*“Usar os dados para prever o nível de impureza no concentrado e apoiar decisões operacionais”*\
\
**Alcançado de forma exploratória e com limitações claras**\
\
Os resultados indicam que há **sinal preditivo real** nos dados de processo. Modelos interpretáveis (OLS e regularizados) superam o\
baseline ingênuo e variáveis como dosagem de reagentes, densidade da polpa e fluxos de ar aparecem consistentemente associadas ao teor\
de sílica.\
\
Por outro lado:

* o poder explicativo global é limitado (R² moderado);
* há evidências de multicolinearidade e regimes operacionais distintos;
* o modelo não deve ser interpretado como sistema de controle, mas como **ferramenta de apoio à decisão**.

**Conclusão:** o objetivo é **atingível como prova de conceito**, com potencial de impacto prático se integrado a um ciclo operacional mais amplo.


# 3. Síntese do Pipeline Analítico

O raciocínio desenvolvido ao longo do projeto seguiu uma progressão lógica:

1. **Entendimento do processo e do contexto industrial**
2. **Validação da qualidade e estrutura temporal dos dados**
3. **Condicionamento e agregação coerente com o processo físico**
4. **Análise exploratória temporal e estatística**
5. **Engenharia de atributos baseada em defasagens horárias**
6. **Comparação entre modelos simples e interpretáveis**
7. **Avaliação sob a ótica de negócio**

Essa estrutura garantiu que **decisões de modelagem fossem sustentadas por evidências**, e não apenas por desempenho numérico.

# 4. Avaliação Comparativa dos Modelos

Foram avaliadas três classes principais de modelos:

## 4.1 Baseline — Persistência (Naive)

* Estratégia: prever que a próxima hora terá o mesmo valor da última observação disponível.
* Métrica principal: **MAE ≈ 1.15**
* Interpretação:

  * Explora exclusivamente a forte autocorrelação temporal;
  * Serve como referência mínima de desempenho;
  * Qualquer modelo útil deve superar esse patamar.

### 4.2 Regressão Linear (OLS)

* Variáveis: médias e desvios padrão horários com defasagens de 1h, 2h e 3h;
* Observações: ~2.800 amostras horárias;
* Resultado:

  * **MAE ≈ 0.90**
  * Ganho absoluto ≈ **0.25 p.p.**
  * Ganho relativo ≈ **22%** em relação ao baseline
* Diagnósticos:

  * Multicolinearidade elevada (Condition Number alto);
  * Resíduos não normalmente distribuídos;
  * Forte autocorrelação residual (Durbin-Watson < 1).

Apesar das limitações estatísticas, o modelo apresentou **ganho consistente e relevante** frente ao baseline.

## 4.3 Modelos Regularizados (Ridge e Lasso)

* Objetivo: mitigar multicolinearidade e reduzir dimensionalidade;
* Resultados:

  * Ridge: estabilização dos coeficientes, desempenho semelhante ao OLS;
  * Lasso: seleção agressiva de variáveis, redução do espaço de atributos;
* Observação central:

  * A regularização **não melhora drasticamente o erro**, mas **aumenta a robustez e interpretabilidade**.

As features sobreviventes ao Lasso indicam **quais sinais do processo carregam informação persistente sobre a sílica**.

# 5. Análise das Hipóteses Iniciais

## Hipótese 1

*“Variáveis de processo medidas ao longo do tempo possuem relação estatística com a qualidade final do concentrado.”*

**Confirmada parcialmente.**\
\
Os modelos desenvolvidos (OLS e versões regularizadas) superam consistentemente o baseline de persistência, indicando\
que variáveis operacionais carregam informação adicional sobre a sílica no concentrado. No entanto, o ganho observado\
é moderado, o que sugere que essas relações existem, mas são **ruidosas, não lineares e influenciadas por fatores não observados**.

## Hipótese 2

*“Existe defasagem temporal entre mudanças no processo e impacto na qualidade, o que sugere que variáveis defasadas (lags) podem ser informativas.*\
*(Isto é, uma flutuação em uma variável pode ter efeito retardado sobre a qualidade, provavelmente o amina flow).”*

**Fortemente confirmada.**\
\
Há atributos defasados que possuem correlação significante com os targets % Iron Concentrate e % Silica Concentrate. Além disso
a análise de autocorrelação da série horária do target evidencia forte dependência temporal. Além disso, variáveis agregadas em janelas horárias\
e defasadas (1h–3h) mostraram-se informativas nos modelos. Isso é coerente com o funcionamento físico do processo, no qual ajustes operacionais levam\
tempo até se refletirem na qualidade aferida em laboratório.

## Hipótese 3

*“O processo pode operar em diferentes regimes ao longo do tempo, afetando a estabilidade das relações observadas.”*

**Confirmada por evidências indiretas.**\
\
A análise exploratória revelou mudanças no padrão de variabilidade ao longo do tempo, heterocedasticidade nos resíduos, e variações na\
correlação local entre ferro e sílica.\
Embora regimes não tenham sido modelados explicitamente, os sinais observados indicam que **as relações entre variáveis não são estacionárias**,\
reforçando a plausibilidade dessa hipótese.

## Hipótese 4

*“Nem todas as variáveis disponíveis são igualmente relevantes; pode haver redundância ou variáveis com baixo poder informativo.”*

**Confirmado.**\
Há atributos (como os de fluxo de ar nas colunas) que apresentam comportamento similar com seus pares.
A análise de correlação, VIF e, principalmente, a aplicação de regularização via Lasso evidenciaram forte redundância entre atributos\
derivados (lags, médias e desvios). O modelo regularizado manteve apenas um subconjunto reduzido de variáveis com poder explicativo\
consistente, validando a necessidade de seleção de features.

# 6. Implicações Operacionais

Os resultados indicam que:

* A qualidade do concentrado responde a **ajustes operacionais recentes**, especialmente em:

  * Vazão e variabilidade de reagentes;
  * Densidade da polpa;
  * Fluxos de ar em colunas específicas.
* O processo apresenta **inércia temporal significativa**, abrindo espaço para:

  * Controle antecipativo;
  * Alertas de degradação de qualidade;
  * Apoio à decisão com horizonte de 1–3 horas.

Os modelos **não substitui operadores**, mas pode **complementar decisões** em ambientes de alta complexidade.

# 7. Limitações do Estudo

* A variável alvo é inserida manualmente e em baixa frequência;
* Não foram modelados explicitamente regimes operacionais;
* Não há variáveis exógenas (ex.: tipo de minério, manutenção);
* Avaliação focada em erro médio, não em risco extremo.

Essas limitações **não invalidam os resultados**, mas delimitam seu escopo de aplicação.

# 8. Revelações para o Negócio

* Modelos simples já capturam parte relevante da dinâmica → alto retorno inicial;
* Regularização é essencial para confiabilidade analítica;
* A sílica é um alvo mais informativo do que o teor de ferro;
* O maior risco não é o erro médio, mas **mudanças silenciosas de regime**;
* Há forte motivação para monitoramento contínuo e detecção de drift.

# 9. Próximos Passos Recomendados

* Modelagem explícita de regimes (clustering temporal);
* Avaliação por faixas críticas de qualidade;
* Testes com modelos não lineares interpretáveis;
* Integração com sistemas de monitoramento em tempo real.

# 10. Conclusão

O estudo demonstra que **é viável antecipar a qualidade do concentrado com base em dados operacionais**, com ganhos mensuráveis sobre abordagens ingênuas.\
Embora existam limitações estruturais, os resultados sustentam o uso do modelo como **ferramenta de apoio à decisão**, e não apenas como exercício analítico.